In [1]:
!nvidia-smi

Tue Jul 11 23:17:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:C4:00.0 Off |                    0 |
| N/A   33C    P0    61W / 400W |      0MiB / 81920MiB |      0%   E. Process |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
import os
from rosemary import jpt_setup; jpt_setup()

from llm.submit import multiline_to_singleline, submit_job_ccc, get_run_statistics
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:

cols = ['model', 'size', 'mixed-precision', 'deepspeed', 
        'gpu mem usage (GB)', 'per-epoch time (hr)', 'per-iter time (s)']

# 55k data points, batch_size=128
data_oasst1 = [
    ('gpt2', 0.124, 'bf16', 'no', 10, None, None),
    ('gpt2-Large', 0.774, 'bf16', 'no', 36, 2.5, 11),
    # incorporate deep speed is costly!
    ('gpt2-Large', 0.774, 'bf16', 'stage 3 no offloading', 40, 6, 25),
    # 1 a100_40g: without offloading OOM on `.backward()`, runs fine with offloading.
    ('gpt2-xl', 1.5, 'bf16', 'stage 3 with offloading', 40, 13, 55),
    # 4 v100_32g: without offloading.
]

df_oasst1 = pd.DataFrame(data_oasst1, columns=cols)

cols = ['model', 'size', 'mixed-precision', 'deepspeed', 
        'gpu mem (GB)', 'cpu mem (GB)', 'per-epoch time (hr)', 'per-iter time (s)']
data = [
    ('gpt2-Large', 0.774, 'bf16', 'no', 36, 7, 9.5, 9),
]

print('instruction tune human-mix on 1 a100_40g:')
df = pd.DataFrame(data, columns=cols)
df

instruction tune human-mix on 1 a100_40g:


,model,size,mixed-precision,deepspeed,gpu mem (GB),cpu mem (GB),per-epoch time (hr),per-iter time (s)
0,gpt2-Large,0.774,bf16,no,36,7,9.5,9


# Finetuning with openinstruct/finetune.py


In [5]:
shell_scripts_template = """
export OPENAI_API_KEY=$(cat ~/.openai_api_key)
export HF_HOME="/dccstor/mit_fm/wpq/hf_cache/"

source /dccstor/mit_fm/miniconda/bin/activate open-instruct
cd /dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/

echo "Running on $(hostname)"
echo "======"
echo "{cmd}"
echo "======"

{cmd}
"""

# [ ! -f "{log_dir}/${{LSB_JOBID}}.out" ] || mv "{log_dir}/${{LSB_JOBID}}.out" "{save_dir}"

In [ ]:
job_name = 'ft'
test_run = True

queue = 'x86_12h' # 'x86_12h'
num_cpus = 20
num_gpus = 2
cpu_mem = 32
require = 'a100_80gb'

# model_name_or_path = 'mosaicml/mpt-7b'; max_seq_length = 2048
# model_name_or_path = 'gpt2'; max_seq_length = 1024
# model_name_or_path = 'gpt2-Large'; max_seq_length = 1024
# model_name_or_path = 'gpt2-xl'; max_seq_length = 1024
model_name_or_path = 'huggyllama/llama-7b'; max_seq_length = 2048


train_file = 'data/processed/oasst1/oasst1_data.jsonl'; train_file_short = 'oasst1'
train_file = 'data/processed/flanv2_cot_oasst1_dolly.jsonl'; train_file_short = 'human_mix'
# train_file = 'data/processed/flanv2_cot_oasst1_dolly_shuffled.jsonl'; train_file_short = 'human_mix_shuffled'

output_dir = f"results/{model_name_or_path.replace('/', ':')}_{train_file_short}"

use_deepspeed = False
# deepspeed_config_file = 'ds_configs/stage3_no_offloading_accelerate_setauto.conf'
# deepspeed_config_file = 'ds_configs/stage3_offloading_accelerate.conf'
deepspeed_config_file = 'ds_configs/stage3_offloading_accelerate_setauto.conf'

use_lora = True
lora_rank = 4
lora_alpha = lora_rank
lora_dropout = 0.05

batch_size_per_gpu = 1
total_batch_size = 128
mixed_precision = 'bf16' # 'bf16', 'fp16'
checkpointing_steps = None # every n steps, where n='1' or every 'epoch'

gradient_acc_steps = int(total_batch_size/num_gpus/batch_size_per_gpu)

print(f"Training {model_name_or_path} "
      f"using {num_gpus} GPUs, "
      f"{batch_size_per_gpu} batch size per GPU, "
      f"{gradient_acc_steps} gradient accumulation steps.")

# do use fast tokenizer since mpt-7b does not have a fast tokenizer counter-part
#     --use_slow_tokenizer \
# do not use flash attention, since having problem installing flash-attn with cuda 12.1
#     --use_flash_attn \

cmd = f"""
{'!cd .. && ' if test_run else ''}accelerate launch \
    --mixed_precision {mixed_precision} \
    --num_machines 1 \
    --num_processes {num_gpus} \
    {'--use_deepspeed' if use_deepspeed else ''}
    {'--deepspeed_config_file '+deepspeed_config_file if use_deepspeed else ''}
    open_instruct/finetune.py \
    --model_name_or_path {model_name_or_path} \
    --tokenizer_name {model_name_or_path} \
    --train_file {train_file} \
    --max_seq_length {max_seq_length} \
    {'--use_lora' if use_lora else ''}
    --lora_rank {lora_rank} \
    --lora_alpha {lora_alpha} \
    --lora_dropout {lora_dropout} \
    --preprocessing_num_workers 16 \
    --per_device_train_batch_size {batch_size_per_gpu} \
    --gradient_accumulation_steps {gradient_acc_steps} \
    --learning_rate 2e-5 \
    --lr_scheduler_type linear \
    --warmup_ratio 0.03 \
    --weight_decay 0. \
    --num_train_epochs 2 \
    --output_dir {output_dir} \
    --with_tracking \
    --report_to tensorboard \
    {'--checkpointing_steps '+str(checkpointing_steps) if checkpointing_steps else ''}
    --logging_steps 1
"""

# things to test to see its effects on (1) eval perf (2) runtime.
#
# - int8
# - mixed_precision bf16 or no
# - with/without LoRA
# - LoRA's rank/alpha (alpha typically set to 2*rank)
# - batch size
# - micro-batch size (largest without running out of memory)


cmd = multiline_to_singleline(cmd)
if test_run:
    print()
    print(cmd)

shell_scripts = shell_scripts_template.format(
    cmd=cmd,
    log_dir=os.getcwd(),
    save_dir=output_dir)
out = submit_job_ccc(
    shell_scripts, 
    job_name=job_name, 
    queue=queue,
    num_cpus=num_cpus,
    cpu_mem=cpu_mem,
    require=require,
    num_gpus=num_gpus,
    test_run=test_run,
)
if not test_run:
    print(out)

In [8]:
# llama7b+lora, micro-bsz=1, bsz=128, 
#     66gb gpu mem, in it 28gb for torch tensor. so fit on 1 a100_80gb

#   1%|▏         | 57/4222 [22:24<25:53:47, 22.38s/it]07/11/2023 23:13:55 - INFO - 
# __main__ -   Step: 57, LR: 9.047619047619049e-06, Loss: 2.2924644947052
# 


!cd .. && accelerate launch --mixed_precision bf16 --num_machines 1 --num_processes 1 open_instruct/finetune.py --model_name_or_path huggyllama/llama-7b --tokenizer_name huggyllama/llama-7b --train_file data/processed/flanv2_cot_oasst1_dolly.jsonl --max_seq_length 2048 --lora_rank 4 --lora_alpha 4 --lora_dropout 0.05 --preprocessing_num_workers 16 --per_device_train_batch_size 1 --gradient_accumulation_steps 128 --learning_rate 2e-5 --lr_scheduler_type linear --warmup_ratio 0.03 --weight_decay 0. --num_train_epochs 2 --output_dir results/huggyllama:llama-7b_human_mix --with_tracking --report_to tensorboard --logging_steps 1





The following values were not passed to `accelerate launch` and had defaults used instead:
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /dccstor/mit_fm/miniconda/envs/open-instruct did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/opt/ibm/lsfsuite/ext/ppm/10.2/linux2.6-glibc2.3-x86_64/lib')}
  warn(msg)

loading file tokenizer.model from cache at /dccstor/mit_fm/wpq/hf_cache/hub/models--huggyllama--llama-7b/snapshots/8416d3fefb0cb3ff5775a7b13c1692d10ff1aa16/tokenizer.model
loading file tokenizer.json from cache at /dccstor/mit_fm/wpq/hf_cache/hub/models--huggyllama--llama-7b/snapshots/8416d3fefb0cb3ff5775a7b13c1692d10ff1aa16/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /dccstor/mit_fm/wpq/hf_cache/hub/models--huggyllama--llama-7b/snapshots/8416d3fefb0cb3ff5775a7b13c1692d10ff1aa16/special_tokens_map.json
loading file tokenizer_config.json from cache at /dccstor/mit_fm/wpq/hf_cache/hub/models--huggyllama--llama-7b/snapshots/8416d3fefb0cb3ff5775a7b13c1692d10ff1aa16/tokenizer_config.json
loading weights file model.safetensors from cache at /dccstor/mit_fm/wpq/hf_cache/hub/models--huggyllama--llama-7b/snapshots/8416d3fefb0cb3ff5775a7b13c1692d10ff1aa16/model.safetensors.index.json
Instantiating LlamaForCausalLM model unde

07/11/2023 23:22:49 - INFO - __main__ - ***** Running training *****
07/11/2023 23:22:49 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) =  Num examples = 270152
07/11/2023 23:22:49 - INFO - __main__ -   Num Epochs = 2
07/11/2023 23:22:49 - INFO - __main__ -   Instantaneous batch size per device = 1
07/11/2023 23:22:49 - INFO - __main__ -  128
07/11/2023 23:22:49 - INFO - __main__ -   Gradient Accumulation steps = 128
07/11/2023 23:22:49 - INFO - __main__ -   Total optimization steps = 4222
  0%|                                                  | 0/4222 [00:00<?, ?it/s]before train loop:
GPU memory occupied: 14739 MB.
torch.cuda.memory_allocated():  13543972864
model.device: cuda:0
model.dtype: torch.bfloat16
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
0  batch.input_

44  batch.input_ids:  torch.Size([1, 1203])
GPU memory occupied: 74411 MB.
torch.cuda.memory_allocated():  27052738560
model.device: cuda:0
model.dtype: torch.bfloat16
45  batch.input_ids:  torch.Size([1, 1029])
GPU memory occupied: 74411 MB.
torch.cuda.memory_allocated():  27191876096
model.device: cuda:0
model.dtype: torch.bfloat16
46  batch.input_ids:  torch.Size([1, 98])
GPU memory occupied: 74411 MB.
torch.cuda.memory_allocated():  27169580544
model.device: cuda:0
model.dtype: torch.bfloat16
47  batch.input_ids:  torch.Size([1, 253])
GPU memory occupied: 74411 MB.
torch.cuda.memory_allocated():  27050449920
model.device: cuda:0
model.dtype: torch.bfloat16
48  batch.input_ids:  torch.Size([1, 436])
GPU memory occupied: 74411 MB.
torch.cuda.memory_allocated():  27070256640
model.device: cuda:0
model.dtype: torch.bfloat16
49  batch.input_ids:  torch.Size([1, 575])
GPU memory occupied: 74411 MB.
torch.cuda.memory_allocated():  27094497792
model.device: cuda:0
model.dtype: torch.bfloat

94  batch.input_ids:  torch.Size([1, 228])
GPU memory occupied: 75327 MB.
torch.cuda.memory_allocated():  27074348544
model.device: cuda:0
model.dtype: torch.bfloat16
95  batch.input_ids:  torch.Size([1, 774])
GPU memory occupied: 75327 MB.
torch.cuda.memory_allocated():  27067065856
model.device: cuda:0
model.dtype: torch.bfloat16
96  batch.input_ids:  torch.Size([1, 280])
GPU memory occupied: 75327 MB.
torch.cuda.memory_allocated():  27136944128
model.device: cuda:0
model.dtype: torch.bfloat16
97  batch.input_ids:  torch.Size([1, 236])
GPU memory occupied: 75327 MB.
torch.cuda.memory_allocated():  27073708544
model.device: cuda:0
model.dtype: torch.bfloat16
98  batch.input_ids:  torch.Size([1, 1530])
GPU memory occupied: 75327 MB.
torch.cuda.memory_allocated():  27068106752
model.device: cuda:0
model.dtype: torch.bfloat16
99  batch.input_ids:  torch.Size([1, 278])
GPU memory occupied: 75327 MB.
torch.cuda.memory_allocated():  27233714688
model.device: cuda:0
model.dtype: torch.bfloat

# Finetuning with openinstruct/finetune_trainer.py


In [ ]:
# llama7b-lora, micro-bsz=1, lora_rank=4: 66gb gpu mem
# llama7b-lora-int8, micro-bsz=1, lora_rank=4: 13gb gpu mem


In [40]:
job_name = 'ft-trainer'
test_run = False

queue = 'x86_12h' # 'x86_12h'
num_cpus = 32
num_gpus = 1
cpu_mem = 32
require = 'a100_80gb'

save_strategy = 'steps'
save_steps = 200

model_name_or_path = 'huggyllama/llama-7b'; max_seq_length = 2048
train_file = 'data/processed/flanv2_cot_oasst1_dolly.jsonl'; train_file_short = 'human_mix'
output_dir = f"results/{model_name_or_path.replace('/', ':')}_{train_file_short}-trainer"
output_dir += '_savebystep'
if num_gpus != 1:
    output_dir += f'_ngpus={num_gpus}'
if test_run:
    output_dir += '_jpt'

num_train_epochs = 1
batch_size_per_gpu = 2
total_batch_size = 128
mixed_precision = 'bf16' # 'bf16', 'fp16'

load_in_8bit = False

use_lora = True
lora_rank = 4
lora_alpha = lora_rank
lora_dropout = 0.05

gradient_acc_steps = int(total_batch_size/num_gpus/batch_size_per_gpu)

print(f"Training {model_name_or_path} "
      f"using {num_gpus} GPUs, "
      f"{batch_size_per_gpu} batch size per GPU, "
      f"{gradient_acc_steps} gradient accumulation steps.")

cmd = f"""
{'!cd .. && ' if test_run else ''}accelerate launch \
    --mixed_precision {mixed_precision} \
    {'--multi_gpu' if num_gpus>1 else ''} 
    --num_machines 1 \
    --num_processes {num_gpus} \
    open_instruct/finetune_trainer.py \
    --model_name_or_path {model_name_or_path} \
    --tokenizer_name {model_name_or_path} \
    --use_fast_tokenizer True \
    --train_file {train_file} \
    --max_seq_length {max_seq_length} \
    {'--use_lora' if use_lora else ''}
    --lora_rank {lora_rank} \
    --lora_alpha {lora_alpha} \
    --lora_dropout {lora_dropout} \
    {'--load_in_8bit' if load_in_8bit else ''} \
    --do_train \
    --preprocessing_num_workers 16 \
    --per_device_train_batch_size {batch_size_per_gpu} \
    --gradient_accumulation_steps {gradient_acc_steps} \
    --learning_rate 2e-5 \
    --lr_scheduler_type linear \
    --warmup_ratio 0.03 \
    --weight_decay 0. \
    --evaluation_strategy "no" \
    --logging_steps 1 \
    --save_strategy {save_strategy} \
    --save_steps {save_steps} \
    --save_total_limit 1 \
    --num_train_epochs {num_train_epochs} \
    --output_dir {output_dir} \
    --bf16 \
    --tf32 True \
    --overwrite_output_dir \
    --report_to tensorboard \
    --torch_dtype bfloat16 \
    --dataloader_num_workers 8
"""


cmd = multiline_to_singleline(cmd)
if test_run:
    print()
    print(cmd)

shell_scripts = shell_scripts_template.format(
    cmd=cmd,
    log_dir=os.getcwd(),
    save_dir=output_dir)
out = submit_job_ccc(
    shell_scripts, 
    job_name=job_name, 
    queue=queue,
    num_cpus=num_cpus,
    cpu_mem=cpu_mem,
    require=require,
    num_gpus=num_gpus,
    test_run=test_run,
)
if not test_run:
    print(out)


Training huggyllama/llama-7b using 1 GPUs, 2 batch size per GPU, 64 gradient accumulation steps.
{'job_id': 1809817, 'jbsub_cmd': 'jbsub -queue x86_12h -name ft-trainer -mem 32g -cores 1x32+1 -require a100_80gb -out /dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/scripts/%J.out bash -c export OPENAI_API_KEY=$(cat ~/.openai_api_key); export HF_HOME="/dccstor/mit_fm/wpq/hf_cache/"; source /dccstor/mit_fm/miniconda/bin/activate open-instruct; cd /dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/; echo "Running on $(hostname)"; echo "======"; echo "accelerate launch --mixed_precision bf16 --num_machines 1 --num_processes 1 open_instruct/finetune_trainer.py --model_name_or_path huggyllama/llama-7b --tokenizer_name huggyllama/llama-7b --use_fast_tokenizer True --train_file data/processed/flanv2_cot_oasst1_dolly.jsonl --max_seq_length 2048 --use_lora --lora_rank 4 --lora_alpha 4 --lora_dropout 0.05 --do_train --preprocessing_num_workers 16 --per_device_train_batch_size

In [35]:
!cd .. && accelerate launch --mixed_precision bf16 --num_machines 1 --num_processes 1 open_instruct/finetune_trainer.py --model_name_or_path huggyllama/llama-7b --tokenizer_name huggyllama/llama-7b --use_fast_tokenizer True --train_file data/processed/flanv2_cot_oasst1_dolly.jsonl --max_seq_length 2048 --use_lora --lora_rank 4 --lora_alpha 4 --lora_dropout 0.05 --load_in_8bit --do_train --preprocessing_num_workers 16 --per_device_train_batch_size 1 --gradient_accumulation_steps 128 --learning_rate 2e-5 --lr_scheduler_type linear --warmup_ratio 0.03 --weight_decay 0. --evaluation_strategy "no" --logging_steps 1 --save_strategy epoch --save_total_limit 1 --num_train_epochs 1 --output_dir results/huggyllama:llama-7b_human_mix-trainer_jpt --bf16 --tf32 True --overwrite_output_dir --report_to tensorboard --torch_dtype bfloat16 --dataloader_num_workers 8

# 


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /dccstor/mit_fm/miniconda/envs/open-instruct did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/opt/ibm/lsfsuite/ext/ppm/10.2/linux2.6-glibc2.3-x86_64/lib')}
  warn(msg)

07/12/2023 00:51:48 - INFO - datasets.builder - Using custom data configuration default-247ebf1b4910b0d3
07/12/2023 00:51:48 - INFO - datasets.info - Loading Dataset Infos from /dccstor/mit_fm/miniconda/envs/open-instruct/lib/python3.10/site-packages/datasets/packaged_modules/json
07/12/2023 00:51:48 - INFO - datasets.builder - Overwrite dataset info from restored data version if exists.
07/12/2023 00:51:48 - INFO - datasets.info - Loading Dataset info from /dccstor/mit_fm/wpq/hf_cache/datasets/json/default-247ebf1b4910b0d3/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96
07/12/2023 00:51:48 - WARNING - datasets.builder - Found cached dataset json (/dccstor/mit_fm/wpq/hf_cache/datasets/json/default-247ebf1b4910b0d3/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
07/12/2023 00:51:48 - INFO - datasets.info - Loading Dataset info from /dccstor/mit_fm/wpq/hf_cache/datasets/json/default-247ebf1b4910b0d3/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd4

07/12/2023 00:52:31 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /dccstor/mit_fm/wpq/hf_cache/datasets/json/default-247ebf1b4910b0d3/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-fc0eb8d201835f20_*_of_00016.arrow
07/12/2023 00:52:31 - INFO - datasets.arrow_dataset - Concatenating 16 shards
[INFO|trainer.py:399] 2023-07-12 00:52:31,435 >> You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
[INFO|trainer.py:407] 2023-07-12 00:52:31,435 >> The model is loaded in 8-bit precision. To train this model you need to add additional modules inside the model such as adapters using `peft` library and freeze the model weights. Please check  the examples in https://github.com/huggingface/peft for more details.
[INFO|trainer.py:776] 2023-07-12 00:52:31,686 >> The following columns in the training set don't have a corresponding

# eval



In [106]:
shell_scripts_template = """
export OPENAI_API_KEY=$(cat ~/.openai_api_key)
export HF_HOME="/dccstor/mit_fm/wpq/hf_cache/"

source /dccstor/mit_fm/miniconda/bin/activate open-instruct
cd /dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/

echo "Running on $(hostname)"
echo "======"
echo "{cmd}"
echo "======"

{cmd}

[ ! -f "{log_dir}/${{LSB_JOBID}}.out" ] || mv "{log_dir}/${{LSB_JOBID}}.out" "{save_dir}"
"""

In [116]:
if any(x in model_name_or_path for x in ['small', 'base', 'medium', 'large']):
    cpu_mem = 2
elif any(x in model_name_or_path for x in ['3b']):
    cpu_mem = 15
elif any(x in model_name_or_path for x in ['7b', '11b', 'xl', 'xxl']):
    cpu_mem = 64
    

True

In [117]:
job_name = 'eval.gsm'
test_run = False
queue = 'x86_1h'
num_cpus = 10
cpu_mem = 64

models = []
# models += ['t5-small', 't5-base', 't5-large', 't5-3b', 't5-11b']
# models += ['t5-11b']
# models += ['google/flan-t5-small', 'google/flan-t5-base', 'google/flan-t5-large', 'google/flan-t5-xl', 'google/flan-t5-xxl']
# models += ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']
models += ['huggyllama/llama-7b'] # , 'mosaicml/mpt-7b'

models = [os.path.join('results/baselines', x) for x in models]

info = {}
cmds = []
for model_name_or_path in models:
    run_id = model_name_or_path
    save_dir = f'{model_name_or_path}/eval/gsm'
    
    cmd = f"""
    python -m eval.gsm.run_eval \
        --data_dir data/eval/gsm/ \
        --model_name_or_path {model_name_or_path} \
        --save_dir {save_dir} \
        --eval_batch_size 5 \
        --n_shot 8
    """
    cmd = multiline_to_singleline(cmd)
    cmds.append(cmd)
    print(cmd)
    
    # submit
    shell_scripts = shell_scripts_template.format(
        cmd=cmd,
        log_dir=os.getcwd(),
        save_dir=save_dir)
    out = submit_job_ccc(
        shell_scripts, 
        job_name=job_name, 
        queue=queue,
        num_cpus=num_cpus,
        cpu_mem=cpu_mem,
        num_gpus=1,
        test_run=test_run,
    )
#     if test_run: print(out['jbsub_cmd'])
    if not test_run:
        info[model_name_or_path] = out['job_id']
    

python -m eval.gsm.run_eval --data_dir data/eval/gsm/ --model_name_or_path results/baselines/huggyllama/llama-7b --save_dir results/baselines/huggyllama/llama-7b/eval/gsm --eval_batch_size 5 --n_shot 8


In [110]:
info = {}
info['results/baselines/t5-small'] = 1763441
info['results/baselines/t5-base'] = 1763442
info['results/baselines/t5-large'] = 1763443
info['results/baselines/t5-3b'] = 1764783
info['results/baselines/t5-11b'] = 1763445

In [111]:


data = []
for k, job_id in info.items():
    logfile_path = f'{job_id}.out'
    out = get_run_statistics(logfile_path)
    data.append((k.split('/')[-1], job_id, out['cpu_time'], out['avg_mem'], out['max_mem']))
    
import pandas as pd
df = pd.DataFrame(data, columns=['name', 'job_id', 'cpu_time', 'avg_mem', 'max_mem'])
df


,name,job_id,cpu_time,avg_mem,max_mem
0,t5-small,1763441,139.93,0.491738,0.597656
1,t5-base,1763442,258.60,0.729512,0.787109
2,t5-large,1763443,76.73,0.957783,1.317383
3,t5-3b,1764783,751.98,6.689150,11.693359
4,t5-11b,1763445,8.53,5.601807,10.000000
